# Making Plots with JuliaDB and OnlineStats

Plotting in JuliaDB is achieved through [StatPlots.jl](https://github.com/JuliaPlots/StatPlots.jl) and recipes in [OnlineStats](https://github.com/joshday/OnlineStats.jl).

In [1]:
addprocs()
using JuliaDB, OnlineStats, StatPlots
gr()

Plots.GRBackend()

## Plotting with [StatPlots.jl](https://github.com/JuliaPlots/StatPlots.jl)

All the power and flexibility of [Plots](https://github.com/JuliaPlots/Plots.jl) is made available to JuliaDB with StatPlots and the `@df` macro.

<div class="alert alert-warning">
<strong>Warning!</strong>  StatPlots will not work on distributed table/ndsparse objects.
</div>

In [ ]:
t = table(@NT(x = randn(100), y = randn(100)))

@df t scatter(:x, :y)

## Plotting Big Data with `partitionplot`


Suppose we have a billion observations.  Visualizing a large amount of data is hard on 

1. Your GPU
2. Your Brain

The idea behind `partitionplot` is this:  Instead of plotting individual observations, observations on the x-axis are grouped together and a summary of the data is displayed on the y-axis.  We can then make an easier-to-create/easier-to-comprehend plot.

```
partitionplot(table, [x,] y; stat, nparts)
```

Keyword args:
- `stat`: OnlineStat used to summarize the `y` over a section of `x`
    - Default: `Extrema(Type_of_Y)`
- `nparts`: Number of parts to group `x` into
    - Default: `100`

In [ ]:
# diamonds = loadtable("datasets/diamonds.csv"; indexcols = [:carat])
diamonds = loadndsparse("datasets/diamonds.csv"; indexcols = [:carat])

### If only one selection is provided, the x axis is "Number of Observations"

In [ ]:
partitionplot(diamonds, :price)

### If two selections are provided (X and Y)

- X will be adaptively grouped together
- The `stat` of Y will be plotted over each group of X

In [ ]:
partitionplot(diamonds, :carat, :price; stat=Hist(10), color=:blues, nparts=20)

In [ ]:
partitionplot(diamonds, :carat, :depth, stat=Hist(20), nparts=15, color=:blues)

In [ ]:
partitionplot(diamonds, :cut, :color, stat=CountMap(String))

In [ ]:
partitionplot(diamonds, :carat, :price, by=:cut, stat=Extrema(), nparts=50,
    layout=(5,1), link = :all, size=(600,600))

# Missing Data

<div class="alert alert-warning">
<strong>Warning!</strong>  <code>partitionplot</code> automatically skips missing values!
</div>

In [ ]:
using DataValues
t = table(DataValueArray(randn(10^6), rand(Bool, 10^6)), randn(10^6))

In [ ]:
partitionplot(t, 1, 2, stat = Hist(-5:.1:5), color = :viridis)

# Mosaic Plot

In [ ]:
plot(reduce(Mosaic(String,String), diamonds, select = (:cut, :color)))

# PlotNN (plot a numerical X vs. numerical Y)

```
PlotNN(nbins)
```

- Approximate scatterplot 

<div class="alert alert-warning">
<strong>Warning!</strong>  The current implementation is quite slow!  <code>partitionplot</code> with <code>stat=Hist(...)</code> is a better option.
</div>

In [ ]:
plot(reduce(PlotNN(100), diamonds, select = (:carat, :price)))